In [ ]:
import tweepy 
from tweepy import OAuthHandler # to authenticate Twitter API
from tweepy import Stream 
from tweepy.streaming import StreamListener
import socket 
import json 
import pandas as pd
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Twitter developer Credentials to connect to twitter account
access_token = '1294388428575047689-uQDkJfSiCikiKi1Yi25NvoFh1SKPvT'
access_secret = 'FWBHK9tgHXUzTvIljhwOrCbBn1aIrOD9VLNGigUXbRBzg'
consumer_key = 'HhzYCY5pUaJII2gUAvzqSVyX3'
consumer_secret = 'UqbEdCradZDBEvWCxQCNTwjhJ1szw10iv9JVHmuaAjSZMR49yl'
records_per_file = 5000  # The number of tweets you want to store per file


class TweetsListener(StreamListener):
    # initialized the constructor
    def __init__(self, csocket):
        self.client_socket = csocket

    def on_data(self, data):
        try:
            # read the Twitter data which comes as a JSON format
            msg = json.loads(data)
            tweet = msg["text"] 
            # the 'text' in the JSON file contains the actual tweet.
            print(msg['text'].encode('utf-8'))
            
            #write to file
            with open("/Users/rashidesai/Downloads/streamoftweetsoutput.csv","a") as fid:
                fid.write(tweet+"\n")
            return True
    
            # the actual tweet data is sent to the client socket
            self.client_socket.send(msg['csv'].encode('utf-8'))
            return True

        except BaseException as e:
            # Error handling
            print("Ahh! Look what is wrong : %s" % str(e))
            return True

    def on_error(self, status):
        print(status)
        return True

def sendData(c_socket):
    # authentication
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # twitter_stream will get the actual live tweet data
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    # filter the tweet feeds related to "hospital"
    twitter_stream.filter(track=['hospital'])
    twitter_stream.filter(count=1000)

# create a socket object
s = socket.socket()

# SparkContext(“local[1]”) would not work with Streaming bc 2 threads are required
sc = SparkContext("local[2]", "Twitter Demo")
ssc = StreamingContext(sc, 10) #10 is the batch interval in seconds

# Get local machine name : host and port
host = "127.0.0.1"
port = 5050
lines = ssc.socketTextStream(host, port)

# Bind to the port
s.bind((host, port))
print("Listening on port: %s" % str(port))

# Wait and Establish the connection with client.
s.listen(5)
c, addr = s.accept()

print("Received request from: " + str(addr))

# Keep the stream data available
sendData(c)

ssc.start()
ssc.stop()

Listening on port: 5050
Received request from: ('127.0.0.1', 52940)
b'@rafadelolmog @hugomabarca @telemadrid https://t.co/RC6lB7dCus'
b'@AntonioMaestre Esta periodista est\xc3\xa1 sentenciada, ir\xc3\xa1 al hospital de infectada proclive a no salir del coma.'
b'RT @_lauryncalo22: all my people in the valley/LA, if u see bro, say sum. My cousin is in the hospital without an eye &amp; other serious healt\xe2\x80\xa6'
b"RT @WaterVole: Susan Kerwin's bat hospital is so inspiring. Full of admiration for what she is doing and achieving @naturalistdara!"
b'RT @ChidiOdinkalu: Folorunsho Olabisi was listed missing following the #LekkiMassacre. \n\nHis body was found by his relatives at the mortuar\xe2\x80\xa6'
b'@is_salsu @julietogechi941 I was at a hospital when ABA protest turned bloody, I saw more than 15 people that were\xe2\x80\xa6 https://t.co/YphOY06vNR'
b'RT @pasanospoco: Telemadrid es ETA'
b'RT @JeremySWallace: NEW: Federal disaster teams are now also heading for El Paso this week to d

b'RT @Den_2042: Lukashenka thugs have badly beaten a 69-years-old protester in Grodno #Belarus. He is now in hospital with head injury. \nv @B\xe2\x80\xa6'
b'RT @JeremySWallace: NEW: Federal disaster teams are now also heading for El Paso this week to deal with the surge in COVID-19 hospitalizati\xe2\x80\xa6'
b'RT @ddale8: As the president says things are going "beautifully" and that we\'re just catching mild cases because we\'re doing so much testin\xe2\x80\xa6'
b'Lo he tenido que ver varias veces porque no me cre\xc3\xada lo que estaba oyendo.\nY se quedan tan pancha, y encima le dice\xe2\x80\xa6 https://t.co/LMB8dqVcQP'
b'RT @ivette_pataki: Despu\xc3\xa9s de 1 mes con ocupaci\xc3\xb3n de 23/40 camas promedio en \xc3\xa1rea COVID en mi hospital, ayer nuevamente tuvimos 40/40. \n\nEl\xe2\x80\xa6'
b'madre m\xc3\xada, es incre\xc3\xadble lo de esta mujer'
b'RT @kurteichenwald: My sister received her bill for the time her husband spent in the hospital with COVID, including almost two wee

b'RT @el_nico17: Un pibe m\xc3\xa1s asesinado x la yuta de @JSchiaretti. Joaqu\xc3\xadn Paredes, estudiante secundario de solo 15 a\xc3\xb1os, trabajaba en la cos\xe2\x80\xa6'
b'RT @BomberosSV: Hicimos maniobras para poder liberar la pierna y rescatar a la joven del tragante de San Miguel, nos apoy\xc3\xb3 personal de @PNC\xe2\x80\xa6'
b'RT @HoustonChron: Federal disaster team sent, overflow hospital set up as COVID surges in El Paso https://t.co/Kq94WBalyI'
b'@NBCNews @NBCNewsTHINK The University of Washington\xe2\x80\x99s Institute for Health Metrics and Evaluation released this wee\xe2\x80\xa6 https://t.co/0z8zcPXSnW'
b'*Pulseira amarela*\n*A morrer de dores*\n*Sangue sai que nem uma torneira*\n\nHospital Santa Maria: \nVamos deixa la ali\xe2\x80\xa6 https://t.co/ETuKZMBJI5'
b'Me ha costado ver el v\xc3\xaddeo entero por la creciente verg\xc3\xbcenza ajena que me ha ido generando.'
b'RT @kerpen: Perhaps unintended, but perfectly foreseeable'
b"@EllerLydia Thanks.  We were lucky.  We

b'RT @nicomartinlpz: DIOS MIO ES QUE TIENE UNA CARA DUR\xc3\x8dSIMA. HAY QUE ECHARLA'
b'RT @_lauryncalo22: all my people in the valley/LA, if u see bro, say sum. My cousin is in the hospital without an eye &amp; other serious healt\xe2\x80\xa6'
b'RT @nto1927: \xe0\xa4\xad\xe0\xa5\x88\xe0\xa4\xaf\xe0\xa4\xbe \xe0\xa4\x98\xe0\xa4\x88 \xe0\xa4\x98\xe0\xa4\x88 \xe0\xa4\xae\xe0\xa5\x87\xe0\xa4\x82 dates \xe0\xa4\xae\xe0\xa5\x87\xe0\xa4\x82 \xe0\xa4\x97\xe0\xa4\xa1\xe0\xa4\xac\xe0\xa4\xa1 \xe0\xa4\x95\xe0\xa4\xb0\xe0\xa4\xa6\xe0\xa5\x80 \xf0\x9f\xa4\xa3 \xe0\xa4\xaf\xe0\xa4\xb9 PR \xe0\xa4\x9f\xe0\xa5\x80\xe0\xa4\xae \xe0\xa4\xb2\xe0\xa5\x87 \xe0\xa4\xa1\xe0\xa5\x82\xe0\xa4\xac\xe0\xa5\x87\xe0\xa4\x97\xe0\xa4\xbe \xe0\xa4\x86\xe0\xa4\xaa\xe0\xa4\x95\xe0\xa5\x8b \xf0\x9f\xa4\xa3 25th September \xe0\xa4\xb9\xe0\xa5\x88\xe0\xa4\x82 excell \xe0\xa4\xae\xe0\xa5\x87\xe0\xa4\x82 and \xe0\xa4\x86\xe0\xa4\x9c 25th October \xe0\xa4\xb9\xe0\xa5\x88. \xe0\xa4\xae\xe0\xa4\xa4\xe0\xa4\xb2\xe0\xa4\xac \xe0

b'RT @_lauryncalo22: all my people in the valley/LA, if u see bro, say sum. My cousin is in the hospital without an eye &amp; other serious healt\xe2\x80\xa6'
b'RT @mworoniak: WPG: fr a front line dr at St. B Hospital. Stay home. \xe2\x80\x9cJust because it\xe2\x80\x99s legal doesn\xe2\x80\x99t mean it\xe2\x80\x99s not really, stupid\xe2\x80\x9d. Explai\xe2\x80\xa6'
b'Y todo as\xc3\xad. Lo \xc3\xbanico que le importa es inaugurar un hospital aunque sea sin personal sanitario.'
b"RT @PAHCBages: Atenci\xc3\xb3!\nEstat d'alarma? Dem\xc3\xa0 tenim dos desnonaments!\n\nEl primer \xc3\xa9s a les 09:30 al carrer hospital n\xc3\xbamero 19-21 tenim el des\xe2\x80\xa6"
b'RT @JeremySWallace: NEW: Federal disaster teams are now also heading for El Paso this week to deal with the surge in COVID-19 hospitalizati\xe2\x80\xa6'
b'RT @maclalal: @atrupar What a f\xe2\x80\x94king lie. My husband is an infectious diseases doc and is working his a$$ off taking care of Covid patients\xe2\x80\xa6'
b'RT @Anton

...

b'RT @AntonioMaestre: Ayuso dando espect\xc3\xa1culo\n\n\xc2\xbfA cu\xc3\xa1ntos sanitarios va a contratar para el nuevo hospital?\n\nA una presidenta no se le pregun\xe2\x80\xa6'
b'RT @P_GomezPerpinya: Lo de Ayuso es de otra liga. Reconoce sin despeinarse que para su hospital de pandemias va a vaciar de personal al res\xe2\x80\xa6'
b'@Frankie43900665 The article has Abbot moving non covid patients to an army hospital and also has state emergency m\xe2\x80\xa6 https://t.co/ZR07JQbDe9'
b'RT @AlexBerenson: LOCKDOWNS KILL. UNNECESSARY MEDIA-DRIVEN PANIC KILLS.'
b"RT @realvictor__: Kid(on hospital bed): Will you visit me when I'm discharged?\n\nNurse: No, I hate graveyards"
b'RT @Gareth_Jenks23: @TegidRoberts @swalesmetroprof Completely flawed analysis of the data\n\nMuch shallower growth curves\nNo exacerbation thi\xe2\x80\xa6'
b'RT @AlexBerenson: LOCKDOWNS KILL. UNNECESSARY MEDIA-DRIVEN PANIC KILLS. https://t.co/NBbIXAAZXZ'
b'RT @AntonioMaestre: Ayuso dando espect\xc3\xa1culo\n\n\xc2\xbf

b'RT @AlexBerenson: LOCKDOWNS KILL. UNNECESSARY MEDIA-DRIVEN PANIC KILLS.'
b'Shocking'
b'@pere_aznar Pues todas!! Acaba de salir la se\xc3\xb1or @IdiazAyuso diciendo que ella no tiene competencias en su nuevo ho\xe2\x80\xa6 https://t.co/tjofvH1Z76'
b'@kumailn Hey!  You do a lot of the heavy lifting to bring up the spirits of folks like me.  I appreciate it.  I wor\xe2\x80\xa6 https://t.co/YtCGNJvJwm'
b"RT @jed_mercurio: For an understanding of respiratory droplet transmission, I'll rely on my six years at medical school and my four years a\xe2\x80\xa6"
b'RT @LoloViejo: -\xc2\xbfCu\xc3\xa1ntos sanitarios va a contratar en el nuevo hospital de pandemias?\n- Ayuso: "Son preguntas que no se le hace a una presi\xe2\x80\xa6'
b'Cada d\xc3\xada est\xc3\xa1 peor.'
b'RT @Irenemate: Yo de verdad que quiero ver de d\xc3\xb3nde van a sacar el personal.'
b'@bcarrebravo https://t.co/6ZWWYZ3xrM'
b"RT @nto1927: Oh they some how found you really? Exactly how that Snehal I'd have no Tweets, not even tagg

b'RT @catswithnohope: Zorro has a serious &amp; rare issue that was found on his xrays His stomach is twisting and can be fatal He is being refer\xe2\x80\xa6'
b'thats not even a blink bruh shut the fuck up'
b'RT @psacera: Esto es lo que el PP llama Libertad de prensa! \xf0\x9f\x98\xa1'
b'RT @PaulNuki: 7/Cygnus made other recommendations. An analysis of NHS surge capacity; a calculation of lives saved/lost by closing hospital\xe2\x80\xa6'
b'RT @_lauryncalo22: all my people in the valley/LA, if u see bro, say sum. My cousin is in the hospital without an eye &amp; other serious healt\xe2\x80\xa6'
b'RT @_lauryncalo22: all my people in the valley/LA, if u see bro, say sum. My cousin is in the hospital without an eye &amp; other serious healt\xe2\x80\xa6'
b'Dectal palo, tal astilla.'
b'RT @KenGardner11: It isn\xe2\x80\x99t just a problem here. It is a problem everywhere \xe2\x80\x94 the consequence of tunnel vision and what happens when raw fea\xe2\x80\xa6'
b'RT @P_GomezPerpinya: Lo de Ayuso

b'PERO MENUDA IN\xc3\x9aTIL.'
b'RT @AntonioMaestre: Ayuso dando espect\xc3\xa1culo\n\n\xc2\xbfA cu\xc3\xa1ntos sanitarios va a contratar para el nuevo hospital?\n\nA una presidenta no se le pregun\xe2\x80\xa6'
b'#AyusoLaIncapaz'
b'RT @hugomabarca: La raz\xc3\xb3n de esta respuesta disparatada es obvia:\n-Lleva meses diciendo que es imposible contratar personal sanitario.\n-Int\xe2\x80\xa6'
b'RT @Conxi_RS: En vez de decir gilipolleces, porque no te vas a un hospital de voluntario, igual cuando lleves meses limpiando v\xc3\xb3mitos, vien\xe2\x80\xa6'
b"@Nmdy60 @BocaGreensFL If you go to my Twitter page it says but I'm a board-certified OB-GYN doctor. I practiced in\xe2\x80\xa6 https://t.co/F8yVlNKjZp"
b'RT @ChidiOdinkalu: Folorunsho Olabisi was listed missing following the #LekkiMassacre. \n\nHis body was found by his relatives at the mortuar\xe2\x80\xa6'
b'RT @TorontoStar: Cancer surgeons in Ontario are reporting a sharp rise in the number of people coming to hospital with advanced canc

b'RT @JeremySWallace: NEW: Federal disaster teams are now also heading for El Paso this week to deal with the surge in COVID-19 hospitalizati\xe2\x80\xa6'
b'RT @AlexBerenson: LOCKDOWNS KILL. UNNECESSARY MEDIA-DRIVEN PANIC KILLS.'
b'#StaySafeStayHealthy'
b'It was only a matter of time b4 one of my family/friends were diagnosed with #covid\xf0\x9f\x98\xa2 My grandfather has tested posi\xe2\x80\xa6 https://t.co/GFEtgR3EHY'
b'@koftownFlower Ibi true, I do my nss for government hospital\nHe come plus his wife \xf0\x9f\x98\x82\xf0\x9f\x98\x82'
b'RT @diasasaigonados: La periodista le pregunta por la contrataci\xc3\xb3n de personal para el nuevo hospital de Valdebebas. Ayuso titubea. Habla d\xe2\x80\xa6'
b'And I personally know of 0 physicians who have been incentivized (or even asked by hospital administrators) to \xe2\x80\x9cove\xe2\x80\xa6 https://t.co/WxunwkbqRv'
b'RT @JeremySWallace: NEW: Federal disaster teams are now also heading for El Paso this week to deal with the surge in COVID-19 